Test Modèle

## Part 7 - Test modèle - Régression logistique

In [67]:
pip install -U scikit-learn    

In [68]:
import pip 
pip.main(["install","matplotlib"]) 

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


0

In [69]:
!pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in c:\python39\lib\site-packages (1.1.2)


You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [70]:
# Chargement des librairies nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [71]:
# Chargement de la dataset
dataset = pd.read_csv("dataset_CV_labelise_features1_200.csv", delimiter = ";", encoding = "utf-8")

In [72]:
dataset.head()
dataset.shape

(5594, 10)

In [73]:
# Basic stats
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 5594

Display of dataset: 


,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha,Grammar,Label
0,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1,0,6,2.80,97.20,"[True, True, True, False, True, True]","['NOUN', 'PROPN', 'VERB', 'NUM', 'PROPN', 'PRO...",1
1,PROFIL,PROFIL,CV_1,1,1,3.27,96.73,[True],['NOUN'],0
2,PERSONNEL Je suis étudiante au lycée Condorcet.,PERSONNEL Je suis étudiante au lycée Condorcet.,CV_1,2,8,7.01,92.99,"[True, True, True, True, True, True, True, False]","['NOUN', 'PRON', 'AUX', 'ADJ', 'ADP', 'NOUN', ...",0
3,Je code depuis l'âge de 13 ans et j'aime créer...,Je code depuis l'âge de 13 ans et j'aime créer...,CV_1,3,24,18.22,81.78,"[True, True, True, False, True, True, False, T...","['PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', ...",0
4,RÉALISATIONS,RÉALISATIONS,CV_1,4,1,18.69,81.31,[True],['NOUN'],0



Basics statistics: 


,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha,Grammar,Label
count,5594,5592,5594,5594.000000,5594.000000,5594.000000,5594.000000,5594,5594,5594.000000
unique,3805,3802,200,NaN,NaN,NaN,NaN,1626,2748,NaN
top,,,CV_10,NaN,NaN,NaN,NaN,[False],['NOUN'],NaN
freq,198,198,66,NaN,NaN,NaN,NaN,638,334,NaN
mean,NaN,NaN,NaN,15.673221,9.132285,50.470518,49.529483,NaN,NaN,0.037898
std,NaN,NaN,NaN,11.268076,10.914849,30.190027,30.190025,NaN,NaN,0.190966
min,NaN,NaN,NaN,0.000000,1.000000,0.240000,0.000000,NaN,NaN,0.000000
25%,NaN,NaN,NaN,7.000000,2.000000,24.190000,23.552500,NaN,NaN,0.000000
50%,NaN,NaN,NaN,14.000000,5.000000,49.530000,50.470000,NaN,NaN,0.000000
75%,NaN,NaN,NaN,23.000000,12.000000,76.447500,75.810000,NaN,NaN,0.000000



Percentage of missing values: 


CV_Sentences           0.000000
Sentences_CV_clean     0.035753
CV_Number              0.000000
Sentence_line          0.000000
Nb_tokens              0.000000
%texte_lu              0.000000
%texte_lu_fin_ligne    0.000000
Is_alpha               0.000000
Grammar                0.000000
Label                  0.000000
dtype: float64

In [74]:
dataset["Label"].unique()
dataset = dataset.loc[(dataset['CV_Sentences'] != "#NOM?"),:]
dataset.shape

(5416, 10)

In [75]:
dataset = dataset.loc[(dataset['CV_Sentences'] != ":"),:]
dataset.shape

(5412, 10)

In [76]:
dataset = dataset.dropna(axis =0, how = 'any')
dataset.shape

(5410, 10)

In [77]:
display(100*dataset.isnull().sum()/dataset.shape[0])

CV_Sentences           0.0
Sentences_CV_clean     0.0
CV_Number              0.0
Sentence_line          0.0
Nb_tokens              0.0
%texte_lu              0.0
%texte_lu_fin_ligne    0.0
Is_alpha               0.0
Grammar                0.0
Label                  0.0
dtype: float64

Multivariate logistic regression

In [78]:
# Separate target variable Y from features X
print("Separating labels from features...")
features_list = ["CV_Sentences","Sentences_CV_clean","CV_Number", "Is_alpha", "Grammar", "Label"]
target_variable = "Label"

X = dataset.drop(features_list, axis = 1)
Y = dataset.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    1
1    0
2    0
3    0
4    0
Name: Label, dtype: int64

X :
   Sentence_line  Nb_tokens  %texte_lu  %texte_lu_fin_ligne
0              0          6       2.80                97.20
1              1          1       3.27                96.73
2              2          8       7.01                92.99
3              3         24      18.22                81.78
4              4          1      18.69                81.31


In [79]:
# Automatically detect positions of numeric/categorical features
idx = 0
numeric_features = []
numeric_indices = []
categorical_features = []
categorical_indices = []
for i,t in X.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
        numeric_indices.append(idx)
    else :
        categorical_features.append(i)
        categorical_indices.append(idx)

    idx = idx + 1

print('Found numeric features ', numeric_features,' at positions ', numeric_indices)
print('Found categorical features ', categorical_features,' at positions ', categorical_indices)

Found numeric features  ['Sentence_line', 'Nb_tokens', '%texte_lu', '%texte_lu_fin_ligne']  at positions  [0, 1, 2, 3]
Found categorical features  []  at positions  []


In [80]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
# WARNING : don't forget stratify=Y for classification problems
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify = Y)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [81]:
# Convert pandas DataFrames to numpy arrays before using scikit-learn
print("Convert pandas DataFrames to numpy arrays...")
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.tolist()
Y_test = Y_test.tolist()
print("...Done")

print(X_train[0:5,:])
print(X_test[0:2,:])
print()
print(Y_train[0:5])
print(Y_test[0:2])

Convert pandas DataFrames to numpy arrays...
...Done
[[25.    1.   45.85 54.15]
 [14.    6.   76.08 23.92]
 [17.    3.   47.06 52.94]
 [11.    1.   27.   73.  ]
 [ 6.    9.   20.78 79.22]]
[[ 3.   15.   35.22 64.78]
 [13.    2.   36.42 63.58]]

[0, 0, 0, 0, 0]
[0, 0]


Training pipeline

In [82]:
# Missing values
print("Imputing missing values...")
print(X_train[0:5,:])
print()
imputer = SimpleImputer(strategy="mean")
X_train[:,numeric_indices] = imputer.fit_transform(X_train[:,numeric_indices])
print("...Done!")
print(X_train[0:5,:]) 
print() 

# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
print()
print(X_train[0:5,:])

# Normalization
numeric_transformer = StandardScaler()

# OHE / dummyfication
categorical_transformer = OneHotEncoder(drop='first')

featureencoder = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_indices),    
        ('num', numeric_transformer, numeric_indices)
        ]
    )

X_train = featureencoder.fit_transform(X_train)
print("...Done")
print(X_train[0:5,:])

# Label encoding
print("Encoding labels...")
print(Y_train[0:5])
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
print("...Done")
print(Y_train[0:5])

Imputing missing values...
[[25.    1.   45.85 54.15]
 [14.    6.   76.08 23.92]
 [17.    3.   47.06 52.94]
 [11.    1.   27.   73.  ]
 [ 6.    9.   20.78 79.22]]

...Done!
[[25.    1.   45.85 54.15]
 [14.    6.   76.08 23.92]
 [17.    3.   47.06 52.94]
 [11.    1.   27.   73.  ]
 [ 6.    9.   20.78 79.22]]

Encoding categorical features and standardizing numerical features...

[[25.    1.   45.85 54.15]
 [14.    6.   76.08 23.92]
 [17.    3.   47.06 52.94]
 [11.    1.   27.   73.  ]
 [ 6.    9.   20.78 79.22]]
...Done
[[ 0.84941168 -0.74490809 -0.14998016  0.14998009]
 [-0.13204091 -0.30377703  0.8477846  -0.84778473]
 [ 0.13562797 -0.56845567 -0.11004316  0.1100431 ]
 [-0.3997098  -0.74490809 -0.77213913  0.7721391 ]
 [-0.84582462 -0.0390984  -0.97743509  0.97743507]]
Encoding labels...
[0, 0, 0, 0, 0]
...Done
[0 0 0 0 0]


In [83]:
# Train model
print("Train model...")
classifier = LogisticRegression()
classifier.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


In [84]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = classifier.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[0 0 0 ... 0 0 0]



Test pipeline

In [85]:
# Missing values
print("Imputing missing values...")
print(X_test[0:5,:])
X_test[:,numeric_indices] = imputer.transform(X_test[:,numeric_indices])
print("...Done!")
print(X_test[0:5,:]) 
print() 

# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
print(X_test[0:5,:])
X_test = featureencoder.transform(X_test)
print("...Done")
print(X_test[0:5,:])

# Label encoding
print("Encoding labels...")
print(Y_test[0:5])
Y_test = encoder.transform(Y_test)
print("...Done")
print(Y_test[0:5])

Imputing missing values...
[[ 3.   15.   35.22 64.78]
 [13.    2.   36.42 63.58]
 [26.    3.   73.29 26.71]
 [ 5.    3.   46.59 53.41]
 [20.    9.   99.46  0.54]]
...Done!
[[ 3.   15.   35.22 64.78]
 [13.    2.   36.42 63.58]
 [26.    3.   73.29 26.71]
 [ 5.    3.   46.59 53.41]
 [20.    9.   99.46  0.54]]

Encoding categorical features and standardizing numerical features...
[[ 3.   15.   35.22 64.78]
 [13.    2.   36.42 63.58]
 [26.    3.   73.29 26.71]
 [ 5.    3.   46.59 53.41]
 [20.    9.   99.46  0.54]]
...Done
[[-1.1134935   0.49025887 -0.50083161  0.50083157]
 [-0.22126388 -0.65668188 -0.46122467  0.46122463]
 [ 0.93863464 -0.56845567  0.75569847 -0.75569859]
 [-0.93504758 -0.56845567 -0.12555588  0.12555581]
 [ 0.40329686 -0.0390984   1.61945976 -1.61945993]]
Encoding labels...
[0, 0, 0, 0, 0]
...Done
[0 0 0 0 0]


In [86]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = classifier.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[0 0 0 ... 0 0 0]



In [87]:
# Print scores
print("accuracy on training set : ", accuracy_score(Y_train, Y_train_pred))
print("accuracy on test set : ", accuracy_score(Y_test, Y_test_pred))
print()

print("f1-score on training set : ", f1_score(Y_train, Y_train_pred))                                            ## ATTENTION REVOIR F1-SCORE
print("f1-score on test set : ", f1_score(Y_test, Y_test_pred))
print()

accuracy on training set :  0.960720887245841
accuracy on test set :  0.9611829944547134

f1-score on training set :  0.0
f1-score on test set :  0.0



In [88]:
# Visualize ROC curves
probas_train = classifier.predict_proba(X_train)[:,1]
precisions, recalls, thresholds = roc_curve(Y_train, probas_train)
fig = go.Figure(
    data = go.Scatter(
        name = 'train',
        x = recalls, 
        y = precisions, 
        mode = 'lines'
    ),
    layout = go.Layout(
        title = go.layout.Title(text = "ROC curve", x = 0.5),
        xaxis = go.layout.XAxis(title = 'False Positive Rate'),
        yaxis = go.layout.YAxis(title = 'True Positive Rate')
    )
)

probas_test = classifier.predict_proba(X_test)[:,1]
precisions, recalls, thresholds = roc_curve(Y_test, probas_test)
fig.add_trace(go.Scatter(
    name = 'test',
    x = recalls, 
    y = precisions, 
    mode = 'lines'
    )
)
fig.show()